In [ ]:
import pandas as pd

df = pd.read_csv("feedback_dataset.csv")

In [1]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
import torch, transformers, accelerate, ipywidgets

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("ipywidgets:", ipywidgets.__version__)


In [ ]:
label_map = {"negative": 0, "neutral": 1, "positive": 2}
df["label"] = df["label"].map(label_map)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")



In [ ]:
def tokenize_data(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2
)



In [ ]:
import torch

class FeedbackDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenize_data(texts)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FeedbackDataset(train_texts, train_labels)
val_dataset = FeedbackDataset(val_texts, val_labels)


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", 
    num_labels=3, 
    ignore_mismatched_sizes=True
)

In [ ]:
import accelerate
import transformers

print(accelerate.__version__)  # should be >=0.26.0
print(transformers.__version__)  # ideally >=4.30
print(transformers.__file__)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",           # Save every epoch to find the best one
    learning_rate=1e-5,              # Slightly faster learning
    per_device_train_batch_size=8,   # Smaller batches for small data
    num_train_epochs=15,              # More rounds to learn patterns
    weight_decay=0.1,                # Stronger protection against memorization
    load_best_model_at_end=True,     # Keep the best version
    metric_for_best_model="f1",      # Optimize for balance, not just accuracy
    logging_steps=10                 # Log more often since the dataset is small
)

print(training_args)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert the model's raw output (logits) into predicted labels (0, 1, or 2)
    predictions = np.argmax(logits, axis=-1)
    
    # Calculate F1 and Accuracy
    # 'weighted' average is best here because it handles the slight imbalance in your data
    f1 = f1_score(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    
    return {"f1": f1, "accuracy": acc}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # ADD THIS LINE
)

trainer.train()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(val_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

print(classification_report(val_labels, y_pred))


In [ ]:
# 1. Save the model using the standard format (avoids the Windows lock)
model.save_pretrained("sentiment_model_final", safe_serialization=False)

# 2. Save the tokenizer
tokenizer.save_pretrained("sentiment_model_final")

print("Model saved successfully in 'sentiment_model_final'!")

In [2]:
from transformers import pipeline

# 1. Load your model as a sentiment-analysis tool
# Replace "sentiment_model" with the folder name where you saved your model
classifier = pipeline("sentiment-analysis", model="./sentiment_model", tokenizer="distilbert-base-uncased")

# 2. Create some brand new sentences to test
test_sentences = [
    "The class was absolutely wonderful and the examples were clear!", # Expected: Positive
    "I'm not sure how I feel about this module, it was okay.",         # Expected: Neutral
    "This was the best course I have ever taken, the instructor was a genius!" # Expected: Negative
]

# 3. Get predictions
results = classifier(test_sentences)

# 4. Print the results
for text, result in zip(test_sentences, results):
    print(f"Text: {text}")
    print(f"Prediction: {result['label']} (Confidence: {result['score']:.2f})\n")

Device set to use cpu


Text: The class was absolutely wonderful and the examples were clear!
Prediction: LABEL_2 (Confidence: 0.96)

Text: I'm not sure how I feel about this module, it was okay.
Prediction: LABEL_1 (Confidence: 0.89)

Text: This was the best course I have ever taken, the instructor was a genius!
Prediction: LABEL_2 (Confidence: 0.94)



In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # ADD THIS LINE
)

trainer.train()